In [1]:
pip install git+https://github.com/IBM/LNN

  Cloning https://github.com/IBM/LNN to /tmp/pip-req-build-qhtwc74a
  Running command git clone --filter=blob:none --quiet https://github.com/IBM/LNN /tmp/pip-req-build-qhtwc74a
  Resolved https://github.com/IBM/LNN to commit 2382067d91b5cba67ce90817925e6e70bc84f25a
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB

In [2]:
!pip install ipywidgets matplotlib

In [3]:
from lnn import Predicates, Variable, Fact, Implies, Model, World, And
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from ipywidgets import interact, widgets, VBox, HBox, Output
from IPython.display import display, clear_output



In [4]:
def check_pred_of_state(predicate_to_check, obj1, obj2):

    # Get the current state of LeftOf relationships
    state = predicate_to_check.state()

    # Form the tuple key for LeftOf(obj1, obj2)
    key = (obj1, obj2)

    # Check if the key exists in the state dictionary
    if key in state:
        # Return True if LeftOf(obj1, obj2) is Fact.TRUE, False otherwise
        return state[key] == Fact.TRUE
    else:
        # If the key is not found, return False (assuming it's not explicitly set to True)
        return False

In [5]:
model = None
UpOf, DownOf, RightOf, LeftOf, TopLeftOf, TopRightOf, BottomLeftOf, BottomRightOf = None, None, None, None, None, None, None, None

In [6]:
# Import necessary libraries

# Define the initial positions of the objects and their details
objects = {
    1: ('apple', 'green', (1, 1)),
    2: ('apple', 'green', (3, 3)),
    3: ('basket', 'red', (2, 2))
}

output = widgets.Output()



# Function to perform the process for one iteration
def process_objects():
    global model, UpOf, DownOf, RightOf, LeftOf, TopLeftOf, TopRightOf, BottomLeftOf, BottomRightOf  # Declare them as global to modify

    # Clear the model's data before adding new data
    model = Model()

    x, y = Variable('x'), Variable('y')

    Obj = Predicates('Obj', arity=1)
    LeftOf = Predicates('LeftOf', arity=2)
    RightOf = Predicates('RightOf', arity=2)
    UpOf = Predicates('UpOf', arity=2)
    DownOf = Predicates('DownOf', arity=2)
    TopLeftOf = Predicates('TopLeftOf', arity=2)
    TopRightOf = Predicates('TopRightOf', arity=2)
    BottomLeftOf = Predicates('BottomLeftOf', arity=2)
    BottomRightOf = Predicates('BottomRightOf', arity=2)
    model.add_knowledge(LeftOf, RightOf, UpOf, DownOf, TopLeftOf, TopRightOf, BottomLeftOf, BottomRightOf)

    # Define the composite relationships (TopLeftOf, TopRightOf, BottomLeftOf, BottomRightOf)
    top_left_relation = Implies(And(LeftOf(x, y), UpOf(x, y)), TopLeftOf(x, y))
    top_right_relation = Implies(And(RightOf(x, y), UpOf(x, y)), TopRightOf(x, y))
    bottom_left_relation = Implies(And(LeftOf(x, y), DownOf(x, y)), BottomLeftOf(x, y))
    bottom_right_relation = Implies(And(RightOf(x, y), DownOf(x, y)), BottomRightOf(x, y))

    # Add the formulas to the model
    model.add_knowledge(
        top_left_relation,
        top_right_relation,
        bottom_left_relation,
        bottom_right_relation,
        world=World.AXIOM
    )

    # Create a DataFrame from the objects dictionary
    data = {
        'obj. no.': list(objects.keys()),
        'Name': [objects[key][0] for key in objects],
        'Colour': [objects[key][1] for key in objects],
        'center': [objects[key][2] for key in objects]
    }

    df = pd.DataFrame(data)

    # Define the relationships (left of, right of, up of, down of)
    left_relations = {}
    right_relations = {}
    up_relations = {}
    down_relations = {}

    for obj1, (_, _, pos1) in objects.items():
        for obj2, (_, _, pos2) in objects.items():
            if obj1 != obj2:
                if pos1[0] < pos2[0]:  # If obj1 is to the left of obj2
                    left_relations[(str(obj1), str(obj2))] = Fact.TRUE
                    right_relations[(str(obj1), str(obj2))] = Fact.FALSE  # Ensure RightOf(2, 1) is False
                if pos1[0] > pos2[0]:  # If obj1 is to the right of obj2
                    right_relations[(str(obj1), str(obj2))] = Fact.TRUE
                    left_relations[(str(obj1), str(obj2))] = Fact.FALSE  # Ensure LeftOf(2, 1) is False
                if pos1[1] > pos2[1]:  # If obj1 is above obj2
                    up_relations[(str(obj1), str(obj2))] = Fact.TRUE
                    down_relations[(str(obj1), str(obj2))] = Fact.FALSE  # Ensure DownOf(2, 1) is False
                if pos1[1] < pos2[1]:  # If obj1 is below obj2
                    down_relations[(str(obj1), str(obj2))] = Fact.TRUE
                    up_relations[(str(obj1), str(obj2))] = Fact.FALSE  # Ensure UpOf(2, 1) is False

    # Add the relations data to the model
    model.add_data({
        LeftOf: left_relations,
        RightOf: right_relations,
        UpOf: up_relations,
        DownOf: down_relations
    })

    model.infer()

    predicate_name = DownOf
    objcheck1 = '3'
    objcheck2 = '1'
    result = check_pred_of_state(predicate_name, objcheck1, objcheck2)
    print(f"{predicate_name}('{objcheck1}', '{objcheck2}') is {result}")

    predicate_name = DownOf
    objcheck1 = '1'
    objcheck2 = '3'
    result = check_pred_of_state(predicate_name, objcheck1, objcheck2)
    print(f"{predicate_name}('{objcheck1}', '{objcheck2}') is {result}")

    # Print the model knowledge base
    #model.print()
    #model.plot_graph()

# Function to plot the objects and update the table
def plot_objects(apple1_x, apple1_y, apple2_x, apple2_y, basket_x, basket_y):
    with output:
        clear_output(wait=True)
        positions = {
            'apple1': (apple1_x, apple1_y),
            'apple2': (apple2_x, apple2_y),
            'basket': (basket_x, basket_y)
        }

        # Update the positions in the objects dictionary
        objects[1] = ('apple', 'green', positions['apple1'])
        objects[2] = ('apple', 'green', positions['apple2'])
        objects[3] = ('basket', 'red', positions['basket'])

        # Plot the objects
        plt.figure(figsize=(6, 6))
        plt.xlim(0, 5)
        plt.ylim(0, 5)
        plt.plot(apple1_x, apple1_y, 'go', markersize=20)  # Green apple 1
        plt.plot(apple2_x, apple2_y, 'go', markersize=20)  # Green apple 2
        plt.plot(basket_x, basket_y, 'ro', markersize=30)  # Red basket
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('Interactive Object Movement')
        plt.grid(True)
        plt.show()

        # Update and display the table
        update_table()
        process_objects()  # Call the process_objects function to perform operations

# Function to create and display the table
def update_table():
    print("Object Details Table:")
    print(f"{'ID':<5} {'Name':<10} {'Color':<10} {'Position':<20}")
    print("-" * 45)
    for obj_id, obj_details in objects.items():
        name, color, position = obj_details
        print(f"{obj_id:<5} {name:<10} {color:<10} {str(position):<20}")

# Create interactive widgets
apple1_x_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=objects[1][2][0], description='Apple 1 X')
apple1_y_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=objects[1][2][1], description='Apple 1 Y')
apple2_x_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=objects[2][2][0], description='Apple 2 X')
apple2_y_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=objects[2][2][1], description='Apple 2 Y')
basket_x_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=objects[3][2][0], description='Basket X')
basket_y_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=objects[3][2][1], description='Basket Y')

# Combine widgets into a single display
interactive_widget = widgets.interactive(
    plot_objects,
    apple1_x=apple1_x_slider,
    apple1_y=apple1_y_slider,
    apple2_x=apple2_x_slider,
    apple2_y=apple2_y_slider,
    basket_x=basket_x_slider,
    basket_y=basket_y_slider
)

# Display the interactive
# Display the interactive widgets and initial plot
display(VBox([
    apple1_x_slider,
    apple1_y_slider,
    apple2_x_slider,
    apple2_y_slider,
    basket_x_slider,
    basket_y_slider,

    output
]))

# Initial plot and table display
plot_objects(objects[1][2][0], objects[1][2][1], objects[2][2][0], objects[2][2][1], objects[3][2][0], objects[3][2][1])


In [24]:
model.print()


***************************************************************************
                                LNN Model

AXIOM Implies: ((RightOf(0, 1) ∧ DownOf(0, 1)) → BottomRightOf(0, 1)) 
('2', '1')                                                  TRUE (1.0, 1.0)
('1', '2')                                                  TRUE (1.0, 1.0)
('1', '3')                                                  TRUE (1.0, 1.0)
('2', '3')                                                  TRUE (1.0, 1.0)
('3', '1')                                                  TRUE (1.0, 1.0)
('3', '2')                                                  TRUE (1.0, 1.0)

OPEN And: (RightOf(0, 1) ∧ DownOf(0, 1)) 
('2', '1')                                                 FALSE (0.0, 0.0)
('1', '2')                                                 FALSE (0.0, 0.0)
('1', '3')                                                 FALSE (0.0, 0.0)
('2', '3')                                                  TRUE (1.0, 1.0)
('3', 

In [25]:
predicate_name = DownOf
objcheck1 = '1'
objcheck2 = '2'
result = check_pred_of_state(predicate_name, objcheck1, objcheck2)
print(f"{predicate_name}('{objcheck1}', '{objcheck2}') is {result}")

DownOf('1', '2') is True


In [26]:
print(objects)

{1: ('apple', 'green', (1.0, 1.0)), 2: ('apple', 'green', (3.0, 1.3)), 3: ('basket', 'red', (2.0, 2.0))}


In [27]:
# Dictionary to map user input directions to predicates
direction_to_predicate = {
    'up': UpOf,
    'down': DownOf,
    'right': RightOf,
    'left': LeftOf,
    'top-left': TopLeftOf,
    'top-right': TopRightOf,
    'bottom-left': BottomLeftOf,
    'bottom-right': BottomRightOf
}

# Function to extract the correct predicate from the prompt
def get_predicate(prompt):
    words = prompt.split()
    for word in words:
        if word in direction_to_predicate:
            return direction_to_predicate[word]
    return ''

# Prompt given by the user
prompt1 = "put the right apple of the basket on the bag"

# Get the predicate name
predname = get_predicate(prompt1)

# Given values
color1 = ''
name1 = 'apple'
colour2 = ''
name2 = ''
colour3 = ''
name3 = 'apple'

# Construct and print the final sentence
print(f"Put the {color1} {name1} on the {predname} of {colour2} {name2} in the {name3}")


Put the  apple on the RightOf of   in the apple


In [28]:
print(predname)

RightOf


In [29]:

def find_objects(color, name):
    matching_ids = []
    if name == '':
        return matching_ids
    for obj_id, (obj_name, obj_color, _) in objects.items():
        if (name == 'object' or name == obj_name) and (color == 'any' or color == '' or color == obj_color):
            matching_ids.append(obj_id)

    return matching_ids

# Find target object
target_object_ids = find_objects(color1, name1)
# Find reference object
reference_object_ids = find_objects(colour2, name2)
# Find destination object
destination_object_ids = find_objects(colour3, name3)

print(f"Target object IDs for {color1} {name1}: {target_object_ids}")
print(f"Reference object IDs for {colour2} {name2}: {reference_object_ids}")
print(f"Destination object IDs for {colour3} {name3}: {destination_object_ids}")


Target object IDs for  apple: [1, 2]
Reference object IDs for  : []
Destination object IDs for  apple: [1, 2]


In [30]:
import math

In [31]:
def calculate_distance(obj1_id, obj2_id):
    obj1_pos = objects[obj1_id][2]
    obj2_pos = objects[obj2_id][2]
    distance = math.sqrt((obj1_pos[0] - obj2_pos[0])**2 + (obj1_pos[1] - obj2_pos[1])**2)
    return distance

# Function to find closest target object to reference object
def find_closest_target(reference_id, target_ids):
    closest_target_id = None
    min_distance = float('inf')

    for target_id in target_ids:
        distance = calculate_distance(target_id, reference_id)
        if distance < min_distance:
            min_distance = distance
            closest_target_id = target_id

    return closest_target_id

# Iterate over each combination of target and reference IDs
predicate_name = predname
true_target_ids = []

if reference_object_ids:
    for objcheck1 in target_object_ids:
        for objcheck2 in reference_object_ids:
            result = check_pred_of_state(predicate_name, str(objcheck1), str(objcheck2))
            print(f"{predname}('{objcheck1}', '{objcheck2}') is {result}")
            if result:
                true_target_ids.append(objcheck1)
else:
    # If no reference object, apply the predicate logic to all possible pairs of target objects
    for i, objcheck1 in enumerate(target_object_ids):
        for j, objcheck2 in enumerate(target_object_ids):
            if i != j:  # Avoid checking the same object with itself
                result = check_pred_of_state(predicate_name, str(objcheck1), str(objcheck2))
                print(f"{predname}('{objcheck1}', '{objcheck2}') is {result}")
                if result:
                    true_target_ids.append(objcheck1)
                    break  # Break after finding one valid pair

# If multiple targets are true, find the closest one to the reference object
if len(true_target_ids) > 1 and reference_object_ids:
    closest_target_id = find_closest_target(reference_object_ids[0], true_target_ids)
    print(f"Closest Target ID to Reference: {closest_target_id}")
    print(f"Associated Information: {objects[closest_target_id]}")
elif len(true_target_ids) == 1:
    print(f"True Target ID: {true_target_ids[0]}")
    print(f"Associated Information: {objects[true_target_ids[0]]}")
else:
    print("No target objects satisfy the condition.")

RightOf('1', '2') is False
RightOf('2', '1') is True
True Target ID: 2
Associated Information: ('apple', 'green', (3.0, 1.3))
